In [4]:
!pip install qdrant-client
!pip install sentence-transformers
!pip install openai

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 MB 7.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 8.3 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11

In [7]:
import pandas as pd 

from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer
from openai import OpenAI

In [9]:
df = pd.read_csv('./top_rated_wines.csv')
df = df[df['variety'].notna()] # remove any NaN values as it blows up serialization
data = df.sample(700).to_dict('records') # Get only 700 records. More records will make it slower to index
len(data)

700

In [12]:
# 1. Load a pretrained Sentence Transformer model
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings


qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance


In [15]:
# Create collection to store wines
qdrant.recreate_collection(
    collection_name="top_wines",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

/var/folders/3y/5lptrc3107l6vgh7khxqmsyc0000gn/T/ipykernel_87663/3075695827.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [16]:
# vectorize!
qdrant.upload_points(
    collection_name="top_wines",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["notes"]).tolist(),
            payload=doc,
        ) for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)

In [61]:
user_prompt = "Suggest me difference kinds of wines from all over the world"


In [62]:
# Search time for awesome wines!

hits = qdrant.search(
    collection_name="top_wines",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=10
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'name': 'Greenock Creek Alices Shiraz 2001', 'region': 'Barossa Valley, Barossa, South Australia, Australia', 'variety': 'Red Wine', 'rating': 98.0, 'notes': 'Rich and fleshy, with pretty coffee, plum, wild berry and spice notes that are smooth and polished, long and flavorful.  An extremely limited release wine from one of Australia\'s "Cult" wineries.'} score: 0.604052944691432
{'name': "Greenock Creek Alice's Shiraz 2003", 'region': 'Barossa Valley, Barossa, South Australia, Australia', 'variety': 'Red Wine', 'rating': 97.0, 'notes': 'Rich and fleshy, with pretty coffee, plum, wild berry and spice notes that are smooth and polished, long and flavorful.  An extremely limited release wine from one of Australia\'s "Cult" wineries.'} score: 0.604052944691432
{'name': 'Chateau Beychevelle (6-Pack OWC Futures Pre-Sale) 2019', 'region': 'St-Julien, Bordeaux, France', 'variety': 'Red Wine', 'rating': 96.0, 'notes': 'The 2019 wines have good, strong colours, well-defined, fresh fruit, and a

In [63]:

# define a variable to hold the search results
search_results = [hit.payload for hit in hits]
search_results

[{'name': 'Greenock Creek Alices Shiraz 2001',
  'region': 'Barossa Valley, Barossa, South Australia, Australia',
  'variety': 'Red Wine',
  'rating': 98.0,
  'notes': 'Rich and fleshy, with pretty coffee, plum, wild berry and spice notes that are smooth and polished, long and flavorful.  An extremely limited release wine from one of Australia\'s "Cult" wineries.'},
 {'name': "Greenock Creek Alice's Shiraz 2003",
  'region': 'Barossa Valley, Barossa, South Australia, Australia',
  'variety': 'Red Wine',
  'rating': 97.0,
  'notes': 'Rich and fleshy, with pretty coffee, plum, wild berry and spice notes that are smooth and polished, long and flavorful.  An extremely limited release wine from one of Australia\'s "Cult" wineries.'},
 {'name': 'Chateau Beychevelle (6-Pack OWC Futures Pre-Sale) 2019',
  'region': 'St-Julien, Bordeaux, France',
  'variety': 'Red Wine',
  'rating': 96.0,
  'notes': 'The 2019 wines have good, strong colours, well-defined, fresh fruit, and a more moderate alcoho

In [32]:
# If using GPT API; Be sure to set the env variable and then run this line

# client = OpenAI()

In [ ]:
# Run LLM locally with Llamafile
# https://github.com/Mozilla-Ocho/llamafile?tab=readme-ov-file
# https://www.notion.so/Week-3-cc979d673e784975aa37afa507cfdaf9

In [41]:
# Now time to connect to the local large language model
# from openai import OpenAI
client = OpenAI(
    base_url="http://127.0.0.1:8080/v1", # "http://<Your api-server IP>:port"
    api_key = "sk-no-key-required"
)
completion = client.chat.completions.create(
    model="llama.cpp",
    messages=[
        {"role": "system", "content": "You are chatbot, a wine specialist. Your top priority is to help guide users into selecting amazing wine and guide them with their requests."},
        {"role": "user", "content": "Suggest me just the name of an amazing red wine from California"},
        {"role": "assistant", "content": str(search_results)}
    ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content='[{\'name\': \'Kistler Vineyards Kistler Vineyard Pinot Noir 1998\', \'region\': \'Sonoma Valley, Sonoma County, California\', \'variety\': \'Red Wine\', \'rating\': 96.0, \'notes\': \'A concentrated wine, with impressive energy to its red and black fruit flavors and subtle earth tones.\'}, {\'name\': \'Guigal La Turque Cote Rotie 2011\', \'region\': \'Cote Rotie, Rhone, France\', \'variety\': \'Red Wine\', \'rating\': 97.0, \'notes\': \'Red fruits, cherry and blackberry dominate the nose on this wine, which is both elegant and intense. Offers smoky, meaty barbecue notes intermixed with forest floor and coffee beans.\'}, {\'name\': "L\'Aventure Estate Cuvee 2006", \'region\': \'Paso Robles, Central Coast, California\', \'variety\': \'Red Wine\', \'rating\': 97.0, \'notes\': \'"A thrilling, nearly perfect efforts is the 2006 Estate Cuvee. A blend of 49% Syrah, 37% Cabernet Sauvignon, and 4% Petit Verdot, the 2006 Estate Cuvee may be even better. Tasting lik

In [44]:
completion.choices[0].message

ChatCompletionMessage(content='[{\'name\': \'Kistler Vineyards Kistler Vineyard Pinot Noir 1998\', \'region\': \'Sonoma Valley, Sonoma County, California\', \'variety\': \'Red Wine\', \'rating\': 96.0, \'notes\': \'A concentrated wine, with impressive energy to its red and black fruit flavors and subtle earth tones.\'}, {\'name\': \'Guigal La Turque Cote Rotie 2011\', \'region\': \'Cote Rotie, Rhone, France\', \'variety\': \'Red Wine\', \'rating\': 97.0, \'notes\': \'Red fruits, cherry and blackberry dominate the nose on this wine, which is both elegant and intense. Offers smoky, meaty barbecue notes intermixed with forest floor and coffee beans.\'}, {\'name\': "L\'Aventure Estate Cuvee 2006", \'region\': \'Paso Robles, Central Coast, California\', \'variety\': \'Red Wine\', \'rating\': 97.0, \'notes\': \'"A thrilling, nearly perfect efforts is the 2006 Estate Cuvee. A blend of 49% Syrah, 37% Cabernet Sauvignon, and 4% Petit Verdot, the 2006 Estate Cuvee may be even better. Tasting lik

In [68]:
completion = client.chat.completions.create(
    model="llama.cpp",
    messages=[
        {"role": "system", "content": "You are chatbot, a wine specialist. Your top priority is to help guide users into selecting amazing wine and guide them with their requests."},
        {"role": "user", "content": "I have a friend who hates wine from Australia. Please show me the top wine you would recommend that is not from there."},
        {"role": "assistant", "content": str(search_results)}
    ]
)

print(completion.choices[0].message.content)

APIConnectionError: Connection error.